In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from scipy.stats import kendalltau
from scipy.spatial.distance import pdist, squareform

from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from math import sqrt
import matplotlib.pyplot as plt
from pyEDM import *

import warnings
warnings.filterwarnings('ignore')

### 一、Input 資料

1.1.載入資料

In [2]:
data = pd.read_csv('/Users/yitsung/git/MastersThesis/data/TaiwanStockData_Top100_EMA')
ticker_2330 = data[data['ticker']==2330].reset_index(drop=True)
ticker_2330 = ticker_2330.drop(columns=['ticker'])
ticker_2330.head()

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14
0,2021-01-04,530.0,540.0,528.0,536.0,39490.0,454.0,12463.0,-33.0,865.0,2342.0,6.0443,521.295251,518.980386,513.251221,5.729165,3.933239,84.477581
1,2021-01-05,536.0,542.0,535.0,542.0,34839.0,-355.0,2884.0,179.0,-451.0,-1374.0,5.3592,525.437881,522.532126,515.535238,6.996887,4.619674,88.417310
2,2021-01-06,555.0,555.0,541.0,549.0,55614.0,-256.0,5355.0,105.0,-4163.0,1.0,6.9696,530.151835,526.614084,518.179719,8.434365,5.454306,91.005801
3,2021-01-07,554.0,570.0,553.0,565.0,53393.0,2200.0,1671.0,-75.0,2060.0,-402.0,8.7664,537.123278,532.531850,521.861371,10.670478,6.574521,93.325963
4,2021-01-08,580.0,580.0,571.0,580.0,62957.0,-502.0,3278.0,187.0,1176.0,-5041.0,9.0658,545.700404,539.847445,526.412277,13.435169,8.026473,94.939847


In [3]:
def make_action_df(ticker_df, Tp): # 只能用一次, 不然頭會一直被去掉

    ticker_df['bs'] = ticker_df['close'].shift(Tp)
    ticker_df['bs'] = ticker_df['close'] - ticker_df['bs']
    ticker_df = ticker_df.dropna().reset_index(drop=True)
    ticker_df['bs'] = (ticker_df['bs'] >= 0).astype(int) # 選擇只看是0還是1就好

    return ticker_df

ticker_2330 = make_action_df(ticker_df=ticker_2330, Tp=1)
ticker_2330

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,bs
0,2021-01-05,536.0,542.0,535.0,542.0,34839.0,-355.0,2884.0,179.0,-451.0,-1374.0,5.3592,525.437881,522.532126,515.535238,6.996887,4.619674,88.417310,1
1,2021-01-06,555.0,555.0,541.0,549.0,55614.0,-256.0,5355.0,105.0,-4163.0,1.0,6.9696,530.151835,526.614084,518.179719,8.434365,5.454306,91.005801,1
2,2021-01-07,554.0,570.0,553.0,565.0,53393.0,2200.0,1671.0,-75.0,2060.0,-402.0,8.7664,537.123278,532.531850,521.861371,10.670478,6.574521,93.325963,1
3,2021-01-08,580.0,580.0,571.0,580.0,62957.0,-502.0,3278.0,187.0,1176.0,-5041.0,9.0658,545.700404,539.847445,526.412277,13.435169,8.026473,94.939847,1
4,2021-01-11,577.0,584.0,574.0,584.0,52772.0,681.0,-7717.0,351.0,-433.0,-10385.0,9.2596,553.361596,546.650220,530.901193,15.749027,9.642037,95.791563,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,2023-11-27,573.0,577.0,568.0,568.0,20322.0,-112.0,-2153.0,59.0,-56.0,-3554.0,4.1507,573.659169,571.840469,562.945683,8.894786,9.086560,81.069290,0
704,2023-11-28,565.0,576.0,565.0,575.0,26932.0,478.0,3323.0,-98.0,687.0,-416.0,5.1624,573.927335,572.326550,563.838595,8.487955,8.966839,76.500832,1
705,2023-11-29,578.0,579.0,570.0,574.0,27787.0,357.0,-180.0,55.0,-553.0,-2383.0,4.8624,573.941868,572.584004,564.591292,7.992712,8.772014,71.301362,0
706,2023-11-30,576.0,577.0,570.0,577.0,54365.0,-32.0,4730.0,-68.0,-770.0,-155.0,7.5527,574.553494,573.263388,565.510455,7.752933,8.568197,68.146342,1


1.2.轉換成EMA+return data

In [4]:
# 取differ
feature_to_differ = ['financing', 'fi', 'ii']
ticker_2330[feature_to_differ] = ticker_2330[feature_to_differ].diff()

# origi_data
origi_data = ticker_2330.copy() # 用來之後還原答案的

# 把價格變成報酬率
ticker_2330['open'] = (ticker_2330['open'] - ticker_2330['open'].shift(1)) / ticker_2330['open'].shift(1)
ticker_2330['high'] = (ticker_2330['high'] - ticker_2330['high'].shift(1)) / ticker_2330['high'].shift(1)
ticker_2330['low'] = (ticker_2330['low'] - ticker_2330['low'].shift(1)) / ticker_2330['low'].shift(1)
ticker_2330['close'] = (ticker_2330['close'] - ticker_2330['close'].shift(1)) / ticker_2330['close'].shift(1)

ticker_2330.replace([float('inf'), -float('inf')], 0, inplace=True) # 不知道為何有些調整過後會變inf, 要拿掉(應該是連兩天的價格都相同)
ticker_2330 = ticker_2330.dropna().reset_index(drop=True)

ticker_2330.tail()

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,bs
702,2023-11-27,-0.006932,-0.001730,-0.010453,-0.012174,20322.0,-355.0,-1299.0,-11.0,-56.0,-3554.0,4.1507,573.659169,571.840469,562.945683,8.894786,9.086560,81.069290,0
703,2023-11-28,-0.013962,-0.001733,-0.005282,0.012324,26932.0,590.0,5476.0,-157.0,687.0,-416.0,5.1624,573.927335,572.326550,563.838595,8.487955,8.966839,76.500832,1
704,2023-11-29,0.023009,0.005208,0.008850,-0.001739,27787.0,-121.0,-3503.0,153.0,-553.0,-2383.0,4.8624,573.941868,572.584004,564.591292,7.992712,8.772014,71.301362,0
705,2023-11-30,-0.003460,-0.003454,0.000000,0.005226,54365.0,-389.0,4910.0,-123.0,-770.0,-155.0,7.5527,574.553494,573.263388,565.510455,7.752933,8.568197,68.146342,1
706,2023-12-01,-0.005208,0.003466,0.005263,0.003466,28798.0,-228.0,2390.0,-543.0,-1060.0,2874.0,5.4736,575.442796,574.145944,566.509681,7.636263,8.381811,61.941017,1


1.3.切Library和Prediction

In [5]:
Library = ticker_2330[ticker_2330['Date'] < '2023-06-30']
# Prediction = ticker_2330[(ticker_2330['Date'] >= '2023-06-30')&(ticker_2330['Date'] <= '2023-07-10')] # 測試用
Prediction = ticker_2330[(ticker_2330['Date'] >= '2023-06-30')&(ticker_2330['Date'] <= '2023-11-29')] # 到用11/29預測11/30就好
Prediction.tail()

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,bs
700,2023-11-23,-0.003472,-0.001727,0.000000,0.001733,15144.0,-360.0,6706.0,225.0,-218.0,93.0,3.0366,575.092451,572.091233,561.544644,10.546589,8.918779,95.718908,1
701,2023-11-24,0.005226,0.000000,0.000000,-0.005190,12503.0,70.0,-4594.0,323.0,-118.0,-2263.0,2.8318,575.073961,572.538736,562.541337,9.997398,9.134503,90.744592,0
702,2023-11-27,-0.006932,-0.001730,-0.010453,-0.012174,20322.0,-355.0,-1299.0,-11.0,-56.0,-3554.0,4.1507,573.659169,571.840469,562.945683,8.894786,9.086560,81.069290,0
703,2023-11-28,-0.013962,-0.001733,-0.005282,0.012324,26932.0,590.0,5476.0,-157.0,687.0,-416.0,5.1624,573.927335,572.326550,563.838595,8.487955,8.966839,76.500832,1
704,2023-11-29,0.023009,0.005208,0.008850,-0.001739,27787.0,-121.0,-3503.0,153.0,-553.0,-2383.0,4.8624,573.941868,572.584004,564.591292,7.992712,8.772014,71.301362,0


1.4.標準化(Minmax)

In [6]:
def make_data_minmax(Library, Prediction):

    feature_to_standardize = Library.columns.to_list()
    feature_to_standardize.remove(Library.columns[0])  # 排除Date
    feature_to_standardize.remove(Library.columns[4])  # 排除close

    # 處理target以外的feature
    scaler_X = MinMaxScaler() 
    Library[feature_to_standardize] = scaler_X.fit_transform(Library[feature_to_standardize])
    Prediction[feature_to_standardize] = scaler_X.fit_transform(Prediction[feature_to_standardize])

    # 處理target feature
    scaler_y = MinMaxScaler() 
    Library['close'] = scaler_y.fit_transform(Library['close'].values.reshape(-1, 1))
    Prediction['close'] = scaler_y.fit_transform(Prediction['close'].values.reshape(-1, 1))

    return Library, Prediction, scaler_y


In [7]:
Library, Prediction, scaler_y = make_data_minmax(Library=Library, Prediction=Prediction)
Prediction.tail()

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,bs
700,2023-11-23,0.398972,0.443121,0.542125,0.502155,0.110144,0.413587,0.764746,0.504069,0.475585,0.512016,0.156896,0.880560,0.838978,0.663346,0.974181,0.814529,0.957189,1.0
701,2023-11-24,0.510887,0.465478,0.542125,0.401524,0.059634,0.525508,0.554881,0.505327,0.491087,0.444932,0.130393,0.880220,0.847641,0.688277,0.945593,0.825712,0.907446,0.0
702,2023-11-27,0.354455,0.443082,0.386153,0.300018,0.209176,0.414888,0.616076,0.501040,0.500698,0.408172,0.301072,0.854245,0.834124,0.698392,0.888198,0.823227,0.810693,0.0
703,2023-11-28,0.264019,0.443044,0.463315,0.656092,0.335597,0.660854,0.741903,0.499166,0.615874,0.497523,0.431995,0.859169,0.843534,0.720727,0.867021,0.817020,0.765008,1.0
704,2023-11-29,0.739670,0.532899,0.674172,0.451687,0.351949,0.475794,0.575143,0.503145,0.423655,0.441515,0.393172,0.859435,0.848517,0.739555,0.841242,0.806921,0.713014,0.0


1.5.整併預測DataFrame

In [8]:
def concate_pred_data(Library, Prediction, th): # th=-1就是Library

    if th < 0:
        Lib_Pred_df = Library
    
    else:
        row_to_add = Prediction.iloc[th]
        Lib_Pred_df = pd.concat([Library, row_to_add.to_frame().T], ignore_index=True)
        
    # 這種concate方法會有非數值問題, 要這樣修正
    Lib_Pred_df[['open', 'high', 'low', 'close', 'volume', 'financing', 'fi', 'ii', 'di', 'rp', 'capital', 'EMA9', 'EMA12', 'EMA26', 'MACD', 'Signal', 'RSI14', 'bs']] = Lib_Pred_df[['open', 'high', 'low', 'close', 'volume', 'financing', 'fi', 'ii', 'di', 'rp', 'capital', 'EMA9', 'EMA12', 'EMA26', 'MACD', 'Signal', 'RSI14', 'bs']].apply(pd.to_numeric, errors='coerce')
    Lib_Pred_df['Date'] = pd.to_datetime(Lib_Pred_df['Date'])

    return Lib_Pred_df

In [9]:
Lib_Pred_df = concate_pred_data(Library=Library, Prediction=Prediction, th=0)
Lib_Pred_df.tail()

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,bs
596,2023-06-26,0.471196,0.420345,0.518317,0.427449,0.156368,0.542843,0.569898,0.512422,0.321104,0.445355,0.202976,0.707717,0.709761,0.673675,0.650284,0.744644,0.508022,0.0
597,2023-06-27,0.470564,0.462043,0.507310,0.478800,0.103541,0.499237,0.591838,0.484297,0.332064,0.468425,0.138160,0.702469,0.707008,0.676538,0.624890,0.733508,0.453906,0.0
598,2023-06-28,0.624932,0.536797,0.582472,0.520660,0.076768,0.523773,0.592271,0.525210,0.324855,0.459063,0.127218,0.699748,0.705846,0.679815,0.605314,0.720099,0.410194,1.0
599,2023-06-29,0.521750,0.504614,0.550096,0.489247,0.072220,0.547038,0.602760,0.504931,0.385161,0.466145,0.115186,0.696833,0.704279,0.682536,0.586186,0.704976,0.348880,0.0
600,2023-06-30,0.265573,0.376204,0.489769,0.553064,0.467544,0.530453,0.617580,1.000000,0.609983,0.465376,0.565818,0.899545,0.902715,0.769223,0.925248,1.000000,0.343062,1.0


### 二、MDRSmap演算法 (前期)

2.1. 製作可餵入EDM格式的train_feature

In [10]:
def find_train_target_feature(data, target):

    df_columns = list(data.columns)
    train_feature = df_columns.copy()
    train_feature.remove('Date') # 先拿掉日期
    
    formatted_columns = ' '.join(df_columns[1:]) # 變成可以餵給 EDM function 參數 'columns' 的形式 
    train_feature.remove(target) # 再拿掉 target_feature

    return formatted_columns, train_feature

In [11]:
# formatted_columns, train_feature = find_train_target_feature(data=Library, target='bs')
# formatted_columns

2.2. 找出target_feature最佳嵌入維度

In [12]:
def find_target_OED(data, target):

    target_OED = EmbedDimension(dataFrame=data, lib=f'1 {len(data)}', pred=f'{len(data)-21} {len(data)-1}', columns=target, showPlot=False) # 4. lib訓練全部, pred看最後20筆

    target_OED_rho = target_OED['rho'].max()
    target_OED = int(target_OED['E'][target_OED['rho'] == target_OED['rho'].max()].iloc[0])

    return target_OED, target_OED_rho


In [13]:
# target_OED, target_OED_rho = find_target_OED(data=Library, target='bs')
# print(f'target_OED: {target_OED}, target_OED_rho: {target_OED_rho}')

2.3. 找出所有有因果關係的train_feature

In [14]:
def find_rho_sig_df(data, ticker, target, target_OED, train_feature, E_max):
    
    crirho = stats.t.ppf(0.95, len(data) - 1) / (len(data) - 2 + stats.t.ppf(0.95, len(data) - 1) ** 2)
    ccm_libSizes = f'{target_OED+10} {len(data)-10} 10'
    # ccm_libSizes = list(range(10, len(data) + 1, 10)) + [len(data)]  # sequence of library size # original

    rho_sig_df = pd.DataFrame(columns=train_feature)
    for train in train_feature:

        ### 找出該train_feature最好的ccm_OED ###
        ccm_E_termRHO = pd.DataFrame(columns=['E', 'term_rho'])
        """
        這裡假設用 term_rho 來選 ccm_OED
        """
        for e in range(1, E_max+1):
            ccm_result = CCM(dataFrame=data, E=e, columns=train, target=target,
                            libSizes=ccm_libSizes, random=False, showPlot=False)
            # print(e, ccm_result[f'{target}:open'].iloc[-1]) # 有時候會有warning, 測試用
            new_data = {'E': e, 'term_rho': ccm_result[f'{target}:{train}'].iloc[-1]}
            ccm_E_termRHO.loc[len(ccm_E_termRHO)] = new_data

        max_term_rho_index = ccm_E_termRHO['term_rho'].idxmax()
        ccm_OED = ccm_E_termRHO.at[max_term_rho_index, 'E']

        ### 用最好的ccm_OED來做該feature的因果檢定 ###
        ccm_result = CCM(dataFrame=data, E=ccm_OED, columns=train, target=target, 
                        libSizes=ccm_libSizes, random=False, showPlot=False)
        """
        這裡假設用 target:train 、 LibSize 來做 kendalltau 檢定
        """
        ccm_result = ccm_result[['LibSize', f'{target}:{train}']]
        ccm_result[f'{target}:{train}'][ccm_result[f'{target}:{train}'] < 0] = 0
        term_rho = ccm_result[f'{target}:{train}'].iloc[-1]

        tau, p_value = kendalltau(ccm_result['LibSize'], ccm_result[f'{target}:{train}']) # 進行 kendalltau 相關檢定

        alpha = 0.05
        if (p_value < alpha) and (term_rho > crirho): # 顯著相關
            rho_sig_df[train] = [term_rho]

        else: # "不" 顯著相關
            rho_sig_df[train] = [0]

    rho_sig_df.index = pd.Index([f'{ticker}_{target}']) 
    
    return rho_sig_df


In [15]:
# rho_sig_df = find_rho_sig_df(data=Library, ticker=2330, target='bs', target_OED=target_OED, train_feature=train_feature, E_max=10)
# rho_sig_df

2.4. 用有因果關係的train_feature建立Embed_df

In [16]:
def make_Embed_df(data, max_lag, target, rho_sig_df):

    #用有因果關係的 train feature + target feature 製作 Embed_df #
    non_zero_columns = rho_sig_df.loc[:, (rho_sig_df != 0).any(axis = 0)] # 選取值非0的column
    train_feature_ls = list(non_zero_columns.columns)
    formatted_columns = ' '.join(train_feature_ls) # 轉成 EDM column 的 input
    columns_to_lag = formatted_columns + f' {target}' # 加入 target 本身

    Embed_df = Embed(dataFrame=data, E=max_lag, tau=-1, columns=columns_to_lag) # 製作 Embed_df
    Embed_df['Date'] = data['Date'] # 加入Date來看index, 才可以防simplex func的bug
    Embed_df.dropna(inplace=True) # 把包含NaN的資料拿掉
    Embed_df = Embed_df.reset_index(drop=True)
    Embed_df = Embed_df[['Date'] + [col for col in Embed_df.columns if col != 'Date']]

    ML_df_date = Embed_df.copy()
    ML_df_date['Date'] = pd.to_datetime(ML_df_date['Date']) # 將index設為日期
    ML_df_date.set_index('Date', inplace=True)
    ML_df_date = ML_df_date.filter(like="(t-0)") # 只留下(t-0)的column

    return Embed_df, ML_df_date

In [17]:
# Embed_df, ML_df_date = make_Embed_df(data=Lib_Pred_df, max_lag=10, target='bs', rho_sig_df=rho_sig_df)
# Embed_df.tail()

In [18]:
# ML_df_date.tail()

2.5. 用simplex randomsearch找出最佳的view

In [19]:
def make_random_simplex(Embed_df, target, target_OED, kmax, kn):    

    Embed_for_train = Embed_df.drop(columns='Date') # 先把 Date 拿掉
    Embed_for_train = Embed_for_train.drop(columns=f'{target}(t-0)') # 先把 target 拿掉
    train_f_ls = list(Embed_for_train.columns) # train_feature
    train_f_num = len(Embed_for_train.columns) # train_feature 的個數

    rho_feature_view = pd.DataFrame(columns=['rho']) # 創建一個df去紀錄每個隨機view的資料
    new_column = pd.DataFrame(columns=['feature_' + str(i) for i in range(1, target_OED+1)])
    rho_feature_view = pd.concat([rho_feature_view, new_column], axis=1)
    k = 1
    while k <= kmax:
        random_pick_train = np.random.choice(train_f_num, target_OED, replace=False)
        # print(random_pick_train)

        train_f_ls = np.array(train_f_ls) # 變成 array 才可以一次選
        select_train_f = train_f_ls[random_pick_train] # 隨機選到的 train_feature
        formatted_random_columns = ' '.join(select_train_f) # 用成符合 EDM 的資料格式
        # print(formatted_random_columns)

        simp = Simplex(dataFrame=Embed_df, E=target_OED, # ver3: 測試近10 or 20個交易日
                       lib=f'1 {len(Embed_df)}', pred = f'{len(Embed_df)-21} {len(Embed_df)-1}', 
                       columns=formatted_random_columns, target=f'{target}(t-0)',
                       embedded = True, showPlot = False) # 原本是False現在改True
        # print(simp)

        sub_simp = simp[['Observations', 'Predictions']] # 計算rho
        rho = sub_simp['Observations'].corr(sub_simp['Predictions'])

        rho_feature_view.loc[len(rho_feature_view), 'rho'] = rho # 將 view 更新到 rho_feature_view 的 df 中
        rho_feature_view.loc[len(rho_feature_view)-1, rho_feature_view.columns[1:]] = select_train_f
        # print(rho)
        k += 1

    allscore = rho_feature_view.sort_values(by='rho', ascending=False).head(kn)
    allscore = allscore.reset_index(drop=True)

    return allscore

In [20]:
# allscore = make_random_simplex(Embed_df=Embed_df, target='bs', target_OED=target_OED, kmax=10000, kn=5)
# allscore.head()

### 三、MDRSmap演算法 (後期)

3.1. 計算每個時點的(view加權)距離

In [21]:
def compute_view_w_distance(Embed_df, allscore):

    ww = allscore['rho'] / allscore['rho'].sum() # 每個view的權重

    dmatrix_ls = []
    for j in range(allscore.shape[0]):

        view_feature = allscore.iloc[j, 1:] # 選取第j個view的所有feature
        view_feature = np.array(view_feature) # 把所有feature變成array才可以從完整Embed_df中找資料
        view_feature_value = Embed_df[view_feature]
        view_matrix = view_feature_value.to_numpy() # 從df形式變array
        view_matrix = np.vstack(view_matrix) # 這樣才能疊成matrix

        Dx_t2 = pdist(view_matrix, metric='euclidean') * ww[j] # 計算加權距離
        Dx_t2 = squareform(Dx_t2) # 將距離變成squareform
        dmatrix_ls.append(Dx_t2)

    v_w_dmatrix = np.sum(dmatrix_ls, axis=0) # 輸出每個時點的view加權距離

    return v_w_dmatrix

In [22]:
# v_w_dmatrix = compute_view_w_distance(Embed_df=Embed_df, allscore=allscore)
# v_w_dmatrix

3.2.尋找elastic-net最佳參數

In [23]:
# ### test ###
# target = 'bs'
# Tp=1

# ML_df_date_new = ML_df_date.copy()
# ML_df_date_new[f'ans(t-0)'] = ML_df_date_new[f'{target}(t-0)'].shift(-Tp) # step.1: 先將target往前移Tp, 製作y
# # ML_df_date_new = ML_df_date_new.multiply(w_tp, axis=0) # step.2: 再將data乘上距離加權
# ML_df_date_new = ML_df_date_new[:-(Tp+1)] # step.3: 拿掉最後Tp+1個, 因為最後面的data是硬拼上去的
# ML_df_date_new

In [24]:
def find_MDRSmap_param(target, ML_df_date, theta_seq, v_w_dmatrix, Tp):

    result_ls = pd.DataFrame(columns=['Theta', 'Score', 'Param']) # 創建紀錄回測結果的dataframe

    ### 將原始資料乘上空間位置權數 ###
    tp = len(ML_df_date) -1
    tp_distence = v_w_dmatrix[tp] # 第tp個時點離其他時點的距離
    mask = np.ones(len(tp_distence), dtype=bool) # 遮蔽該時點計算平均數
    mask[tp] = False
    dpar = np.mean(tp_distence[mask]) # 第tp個時點離其他時點的平均數

    for theta in theta_seq:
        w_tp = np.exp(-theta * tp_distence / dpar) # 計算每個時點資料的加權
        w_tp = np.sqrt(w_tp)

        ### 加入答案列 ###
        ML_df_date_new = ML_df_date.copy()
        ML_df_date_new[f'ans(t-0)'] = ML_df_date_new[f'{target}(t-0)'].shift(-Tp) # step.1: 先將target往前移Tp, 製作y
        ML_df_date_new = ML_df_date_new.multiply(w_tp, axis=0) # step.2: 再將data乘上距離加權
        ML_df_date_new['ans(t-0)'] = ML_df_date_new['ans(t-0)'].apply(lambda x: 1.0 if x != 0 else x) # step.3: 把ans非0的部分變成1
        ML_df_date_new = ML_df_date_new[:-(Tp+1)] # step.4: 拿掉最後Tp+1個, 因為最後面的data是硬拼上去的
        # ML_df_date_new = ML_df_date_new.drop(columns=[f'{target}(t-0)']) # step.5: 原paper有刪target啦, 這邊可選擇刪或不刪
        # ML_df_date_new = ML_df_date_new.dropna().reset_index(drop=True) # 不確定要不要用

        ### 分拆train, validation(以近60天為基準) ###
        X = ML_df_date_new.iloc[:, :-1]
        y = ML_df_date_new.iloc[:, -1]
        val_fold = [-1] * (len(X)-60) + [0] * 60 # 最後60筆當validation set
        ps = PredefinedSplit(test_fold=val_fold)

        logistic_elastic_net = LogisticRegression(penalty='elasticnet', 
                                                  solver='saga', # 只有saga支持elasticnet
                                                  random_state=87)

        ### grid search ###
        param_grid = {'l1_ratio': [0.9, 0.1, 0.01, 0.001, 0.0001],
                      'C': [0.001, 0.01, 0.1, 1, 10, 100],
                      'tol': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
                      'fit_intercept': [True], 
                      'intercept_scaling': [0.1],
                      'warm_start': [True]}
        grid_search = GridSearchCV(estimator=logistic_elastic_net, 
                                   param_grid=param_grid, 
                                   cv=ps, scoring='accuracy', 
                                   return_train_score=True)       

        grid_search.fit(X, y)

        ### 記錄結果 ###
        result_ls.loc[len(result_ls), 'Theta'] = theta
        result_ls.loc[len(result_ls)-1, 'Score'] = grid_search.best_score_
        result_ls.loc[len(result_ls)-1, 'Param'] = [grid_search.best_params_]

        theta = result_ls['Theta'][result_ls['Score'].idxmax()]
        param = result_ls['Param'][result_ls['Score'].idxmax()][0]

    return result_ls, theta, param

In [25]:
# result_ls, theta, param = find_MDRSmap_param(target='bs', 
#                                              ML_df_date=ML_df_date, 
#                                              theta_seq=[1,2,4,7,11,16,22], 
#                                              v_w_dmatrix=v_w_dmatrix,
#                                              Tp=1)

In [26]:
# result_ls['Param'][5]

3.3.用最佳參數訓練MDRSmap

In [27]:
def MDRSmap_model(target, ML_df_date, theta, v_w_dmatrix, param, Tp):

    ### 將原始資料乘上空間位置權數 ###
    tp = len(ML_df_date) -1
    tp_distence = v_w_dmatrix[tp] # 第tp個時點離其他時點的距離
    mask = np.ones(len(tp_distence), dtype=bool) # 遮蔽該時點計算平均數
    mask[tp] = False
    dpar = np.mean(tp_distence[mask]) # 第tp個時點離其他時點的平均數

    w_tp = np.exp(-theta * tp_distence / dpar) # 計算每個時點資料的加權
    w_tp = np.sqrt(w_tp)

    ### 加入答案列 ###
    ML_df_date_new = ML_df_date.copy()
    ML_df_date_new[f'ans(t-0)'] = ML_df_date_new[f'{target}(t-0)'].shift(-Tp) # step.1: 先將target往前移Tp, 製作y
    ML_df_date_new = ML_df_date_new.multiply(w_tp, axis=0) # step.2: 再將data乘上距離加權
    ML_df_date_new['ans(t-0)'] = ML_df_date_new['ans(t-0)'].apply(lambda x: 1.0 if x != 0 else x) # step.3: 把ans非0的部分變成1
    ML_df_date_new = ML_df_date_new[:-(Tp+1)] # step.4: 拿掉最後Tp+1個, 因為最後面的data是硬拼上去的
    # ML_df_date_new = ML_df_date_new.drop(columns=[f'{target}(t-0)']) # step.5: 原paper有刪target啦, 這邊可選擇刪或不刪
    # ML_df_date_new = ML_df_date_new.dropna().reset_index(drop=True) # 不確定要不要用

    ### 分拆train, validation(以近60天為基準) ###
    X = ML_df_date_new.iloc[:, :-1]
    y = ML_df_date_new.iloc[:, -1]

    logistic_elastic_net = LogisticRegression(penalty='elasticnet', 
                                              solver='saga', # 只有saga支持elasticnet
                                              random_state=87,
                                              **param)
                             

    logistic_elastic_net.fit(X, y)

    return logistic_elastic_net

In [28]:
# logistic_elastic_net = MDRSmap_model(target='bs', ML_df_date=ML_df_date, 
#                                      theta=theta, v_w_dmatrix=v_w_dmatrix, param=param, Tp=1)

3.4.進行預測

In [29]:
# X_pred = np.array(ML_df_date.iloc[-1]).reshape(1, -1)
# y_pred = logistic_elastic_net.predict(X_pred)
# y_pred = y_pred[0]
# y_pred

3.5.製作評估dataframe

In [30]:
# Date = origi_data['Date'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-11-30')].reset_index(drop=True)
# Today = origi_data['bs'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-11-30')].reset_index(drop=True)
# Yesterday = origi_data['bs'][(origi_data['Date']>='2023-06-30')&(origi_data['Date']<='2023-11-29')].reset_index(drop=True)

# MDRSmap_result = pd.DataFrame(Date)
# MDRSmap_result['Observations'] = Today
# MDRSmap_result['Predictions'] = None
# MDRSmap_result['Yesterday'] = Yesterday
# MDRSmap_result

In [31]:
# th=0
# MDRSmap_result.loc[th, 'Predictions'] = y_pred
# MDRSmap_result

In [32]:
# MDRSmap_result['Date'][th]

### 四、完整預測流程

4.1.完整預測流程

In [ ]:
### 一、Input資料(到1.5.前略) ###
### 從 6/30 預測 7/03 開始 ###

### 製作評估dataframe ###
Date = origi_data['Date'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-11-30')].reset_index(drop=True)
Today = origi_data['bs'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-11-30')].reset_index(drop=True)
Yesterday = origi_data['bs'][(origi_data['Date']>='2023-06-30')&(origi_data['Date']<='2023-11-29')].reset_index(drop=True)

MDRSmap_result = pd.DataFrame(Date)
MDRSmap_result['Observations'] = Today
MDRSmap_result['Predictions'] = None
MDRSmap_result['Yesterday'] = Yesterday

### 開始進行MDRSmap演算法 ###
for th in range(0, len(Prediction)):

    if th == 0: ### th = 0, 要找view ###
        Lib_Pred_df = concate_pred_data(Library=Library, Prediction=Prediction, th=th)
        formatted_columns, train_feature = find_train_target_feature(data=Lib_Pred_df, target='bs')
        target_OED, target_OED_rho = find_target_OED(data=Lib_Pred_df, target='bs')
        rho_sig_df = find_rho_sig_df(data=Lib_Pred_df, ticker=2330, target='bs', 
                                     target_OED=target_OED, train_feature=train_feature, E_max=10)
        Embed_df, ML_df_date = make_Embed_df(data=Lib_Pred_df, max_lag=10, target='bs', rho_sig_df=rho_sig_df)
        allscore = make_random_simplex(Embed_df=Embed_df, target='bs', target_OED=target_OED, kmax=10000, kn=5)

        ### 這邊要concate所有股票的Library資料 + 一個Predict ###



        v_w_dmatrix = compute_view_w_distance(Embed_df=Embed_df, allscore=allscore)
        result_ls, theta, param = find_MDRSmap_param(target='bs', 
                                                     ML_df_date=ML_df_date, 
                                                     theta_seq=[1,2,4,7,11,16,22], 
                                                     v_w_dmatrix=v_w_dmatrix,
                                                     Tp=1)
        logistic_elastic_net = MDRSmap_model(target='bs', ML_df_date=ML_df_date, 
                                             theta=theta, v_w_dmatrix=v_w_dmatrix, param=param, Tp=1)
        
        ### 預測 ###
        X_pred = np.array(ML_df_date.iloc[-1]).reshape(1, -1)
        y_pred = logistic_elastic_net.predict(X_pred)
        y_pred = y_pred[0]

        ### 將結果併入評估dataframe ###
        MDRSmap_result.loc[th, 'Predictions'] = y_pred
        print(f"{MDRSmap_result['Date'][th]}: finished")
    
    else: ### th > 0, 僅算距離+預測 ###

        ### 這邊要concate所有股票的Library資料 + 一個Predict ###

        Lib_Pred_df = concate_pred_data(Library=Library, Prediction=Prediction, th=th)
        Embed_df, ML_df_date = make_Embed_df(data=Lib_Pred_df, max_lag=10, target='bs', rho_sig_df=rho_sig_df)
        v_w_dmatrix = compute_view_w_distance(Embed_df=Embed_df, allscore=allscore)
        result_ls, theta, param = find_MDRSmap_param(target='bs', 
                                                     ML_df_date=ML_df_date, 
                                                     theta_seq=[1,2,4,7,11,16,22], 
                                                     v_w_dmatrix=v_w_dmatrix,
                                                     Tp=1)
        logistic_elastic_net = MDRSmap_model(target='bs', ML_df_date=ML_df_date, 
                                             theta=theta, v_w_dmatrix=v_w_dmatrix, param=param, Tp=1)
        
        ### 預測 ###
        X_pred = np.array(ML_df_date.iloc[-1]).reshape(1, -1)
        y_pred = logistic_elastic_net.predict(X_pred)
        y_pred = y_pred[0]

        ### 將結果併入評估dataframe ###
        MDRSmap_result.loc[th, 'Predictions'] = y_pred
        print(f"{MDRSmap_result['Date'][th]}: finished")

4.3.評估結果

In [ ]:
MDRSmap_result.head(60)

,Date,Observations,Predictions,Yesterday
0,2023-07-03,1,1.0,1
1,2023-07-04,1,1.0,1
2,2023-07-05,0,1.0,1
3,2023-07-06,0,1.0,0
4,2023-07-07,1,0.0,0
5,2023-07-10,1,1.0,1
6,2023-07-11,1,1.0,1
7,2023-07-12,1,1.0,1
8,2023-07-13,1,1.0,1
9,2023-07-14,1,1.0,1


In [ ]:
ACC = len(MDRSmap_result[MDRSmap_result['Predictions'] == MDRSmap_result['Observations']]) / len(MDRSmap_result['Observations'])
print('ACC: ', ACC)

ACC:  0.5428571428571428
